# Goal of this notebook:

Use a spectrum with known peaks to calibrate the channels of a detector,
and then use the calibration to plot an unknown spectrum.

To calibrate a spectrum we calculate:
  - dispersion [keV / channel], which is the width of each channel
  - offset [channels], which is the distance from first data entry to keV=0

When we have calibrated a spectrum, we could also find:
  - resolution,
  - quantities of each element, 
  - ... ?


### What we will do:

1. Read in the data of a known spectrum
2. Fit the data to a gaussian
3. Calibrate the x-axis
4. Plot the calibrated data
5. Use the calibrated x-axis on an unknown spectrum from the same source


**Repo avaliable on Github: [https://github.com/brynjarmorka/spectroscopy-channel-calibration](https://github.com/brynjarmorka/spectroscopy-channel-calibration)**

# We will be working with the data stored in a dictionary like this:

~~~python
 spectrum = {
    "name": str                     - the name of the spectrum used on plots,
    "filepath": str                 - (relative) path to the file,
    "channel": list of int          - the uncalibrated channel axis,
    "intensity": list of float      - the intensity at each channel relative to the highest intensity (a.u.),
    "peaks_keV": list of float      - theoretical value in keV of the peaks used in calibration,
    "peaks_names": list of str      - optional naming of peaks,
    "peaks_channel": list of float  - channel value of peaks (first a int guess then fitted to exact float),
    "dispersion": float             - the calibrated channel width in keV/channel,
    "offset": float                 - distance from first channel entry to real zero in channels,
    "kev_calibrated": list of float - the calibrated keV axis, which is (channel - offset) * dispersion,
    "fit_params": list of float     - the fitted peak parameters as [amp1, mu1, std1, amp2, mu2, std2, ...],
    "fit_cov": list of float        - the covariance matrix to the fitted gaussians,
    "intensity_fit": list of float  - the gaussian fitted intensities,
    "start_str": str                - the whole line before the data starts,
    "stop_str": str                 - the whole line after the data ends,
    "line_endings": str             - the line endings to be stripped of the data (eg. '\n'),
    "delimiter": str                - delimiter between x and y data (.emsa files have xy data, not just y)
}
~~~

In [ ]:
# import all you need
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import curve_fit
import json  # to save the data after calibration

# functions imported from helper_files
from helper_files.plotting import plot_lines, plotly_plot
from helper_files.gaussian_fitting import (
    gaussian,
    n_gaussians,
    fit_n_peaks_to_gaussian,
    area_under_peak,
)
from helper_files.calibration import calibrate_channel_width_two_peaks, channel_to_keV
from helper_files.spectrum_dict import (
    init_known_spectrum,
    init_unknown_spectrum_with_known,
)
from helper_files.saving_json import (
    save_spectrum_to_json,
    read_saved_spectrum_from_json,
)

In [ ]:
# this will load the helper modules each time you make changes to them, without having to restart the kernel
%load_ext autoreload
%autoreload 2

### Reading the data into arrays, plotting.py


When making the spectrum dictionary, we need the following variables:


- filepath
    - set the path to the file, or just use the example file
- name
    - just a name of the spectrum
- start_str
    - open the data file and find out where the data starts and stops.
        - in the .emsa file the data starts after "#SPECTRUM    : Spectral Data Starts Here",
- stop_str
    - the .emsa file ends the data with "#ENDOFDATA   : "
- line_endings
    - take note of the line endings,
        - in the .emsa file data the line endings are '\n'
- delimiter, but ONLY if the data file contains both x and y values, 
    - take note of what the data is separated by
        - in the example data it is separated by a comma and a space ", "
        - do not specify the delimiter if the data file only contains y values

You *can* also specify if you already know:

- peaks_keV
    - theoretical value of the two peaks to use in calibration
- peaks_names
    - name of the two peaks, optional
- peaks_channel
    - the peak positions in channels to do gaussian fitting
    - the two first peaks must correspond to the peaks in peaks_keV for the calibration

**If you do not specify the three values above, you must do it later down in the notebook.**


In [ ]:
# reading the files

# you MUST uncomment the right init for the filetype you are going to use.
# that is to ensure that the file is read correctly. 
# If you get an error in this cell, check the init function for the filetype you are using.
# file reading parameters: filepath, start_str, stop_str, delimiter, line_endings

#
# # .msa
#
# SEM known Cu
s_SEM_Cu = init_known_spectrum(
    name="SEM known: Cu",
    filepath="Lab3_data/SEM_known_Cu.msa",
    start_str="#SPECTRUM    : Spectral Data Starts Here",
    stop_str="#ENDOFDATA   : End Of Data and File",
    line_endings=", \n",

    # since I already know the peaks in Cu, I can set the peak positions, the name and the channels
    peaks_keV=[0.9297, 8.0478],  # Cu_Kb=8.9052, can use this the check the calibration
    peaks_names=["Cu_La", "Cu_Ka"],
    peaks_channel=[57, 412, 456],  # we want to fit tree peaks
)

#
# # .emsa
#
s_TEM = init_known_spectrum(
    name=".emsa",
    filepath="Lab3_data/TEM_known_NiO_on_Mo_A.emsa",
    start_str="#SPECTRUM    : Spectral Data Starts Here",
    stop_str="#ENDOFDATA   : ",
    line_endings="\n",
    delimiter=", ",

    # these three needs to be set in cell 10
    peaks_keV=None,
    peaks_names=None,
    peaks_channel=None,
)

#
# # .mca
#
s_XRF = init_known_spectrum(
    name=".mca",
    filepath="Lab3_data/XRF_known_Cu.mca",
    start_str="<<DATA>>",
    stop_str="<<END>>",
    line_endings="\n",

    # these three needs to be set in cell 10
    peaks_keV=None,
    peaks_names=None,
    peaks_channel=None,
)


In [ ]:
# set which spectrum to do the operations on

# s is the spectrum that are used in the following code blocks.
# s is a view, which means that the operations on s are done on the dict it is equal,
# since they are both pointing to the same memory.

# comment in the one you want to run

s = s_SEM_Cu
# s = s_TEM
# s = s_XRF


In [ ]:
# plotting the spectrum with channels as x-axis

fig_intitial = plotly_plot(
    x=s["channel"],
    y=s["intensity"],
    title=s["name"],
    xaxis_title="channel",
    yaxis_title="intensity",
)
fig_intitial.show()

print(
    f"Check if the peaks in s['peaks_channel'] are in the right place in the plot here: {s['peaks_channel']}"
)
print(
    "The two first peaks are the ones we will use for calibration later, ie. the ones we know the energy of."
)
print(
    f"You have specified s['peaks_keV'] as: {s['peaks_keV']} keV, and are named {s['peaks_names']}"
)

### Fittin the peaks to gaussians

##### see the helper file gaussian_fitting.py

Now we find two or more peaks in the data, and we want to fit a gaussian to each of them. 
We need to know what the theoretical value of at least two the guessed peaks, so that we can calibrate the spectrum later.

In the Ex1 we can use Ga_Ka=9.2517 keV and Ga_La=1.098 keV, or we can use As_Ka=10.5336 keV and As_La=1.2819 keV. Or we can use both

eg peak_guesses = [9.2517, 10.5336]

Short overview of the used functions in gaussian_fitting.py
- def gaussian(x, amp, mu, std):
    - the function gaussian defines a gaussian function.

- def n_gaussians(x, *args):
    - since the gaussians could potentially partially overlap, we need to define a function that returns the sum of n gaussians.
        - Eg. like Ga_Kb=10.2642 and As=10.5436 in Ex1

- def fit_n_peaks_to_gaussian(x, raw_y, guessed_peaks, guessed_std=1, guessed_amp=1,):
    - now we need a functions which fits peak guesses to gaussian curves.
    - we will use the scipy.optimize.curve_fit function for this. More info in the function.
    - with normalized counts it usually works nice with guessing all std and amplitues as 1

Additional info:
I tried fitting to the raw keV-valus of the .emsa file, but that did not work. 
I think it is because the x values are not integers, and for some reason that hindered the fitting.
I do suspect that it migh be because the std and amp guesses are way off, but I am not sure.

This is the code that did not work:
~~~
peak_guesses = [0.02, 0.27, 0.97, 1.1, 1.29, 1.75, 9.2517, 10.24, 10.5336, 11.75] 
fit_vals = fit_n_peaks_to_gaussian(data[0], data[1], peak_guesses)
~~~

**Be aware: sometimes the fitting sets a peak as the background. Always inspect the plot**


In [ ]:
# Fitting the data to n gaussian peaks

#
#
# NB! if this cell crash with something like:
# "RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 5000."
# then adjust your peak guesses, or try to fit fewer/more peaks at once.
#
#

# you can fit multiple peaks at once, but we only use two peaks for calibration.
# change s['peaks_channel'] to fit more peaks at once.
# s['peaks_channel'] = [...]


# # comment in and change the lines below if necessary

# these 3 lnes below work for TEM known NiO on Mo_A.emsa
# s["peaks_channel"] = [106, 766]
# s["peaks_names"] = ["Ni_La", "Ni_Ka"]
# s["peaks_keV"] = [.851, 7.478]

# these 3 lnes below work for XRF known Cu.mca
# s["peaks_channel"] = [225, 248]
# s["peaks_names"] = ["Cu_Ka", "Cu_Kb"]
# s["peaks_keV"] = [8.047, 8.905]

# use a different set of numbers if you want to fit another file


if s["peaks_channel"] is None:
    print(
        "ERROR: no peaks specified, please specify the peaks in the spectrum. No fitting was done."
    )
    print(
        "Comment out the line above to set s['peaks_channel'] =[peak1, peak2, ...] manually."
    )
else:
    fit_vals = fit_n_peaks_to_gaussian(
        x=s["channel"],
        y=s["intensity"],
        guessed_peaks=s["peaks_channel"],
    )
    s["fit_params"] = fit_vals[0]
    s["fit_cov"] = fit_vals[1]

    # update the channel values of the peaks
    s["peaks_channel"] = s["fit_params"][1::3]

    # addin the fitted gaussian to the spectrum-dictionary
    s["intensity_fit"] = n_gaussians(s["channel"], *fit_vals[0])


print(f'Fitted peaks at channel: {s["peaks_channel"]}')

In [ ]:
# plot using the following function, see helper_files/plotting.py

fig_fit = plotly_plot(
    s["channel"],
    s["intensity"],
    y_fit=s["intensity_fit"],
    vlines=s["peaks_channel"],
    vlines_name=s["peaks_names"],
    title=f"Gaussian fitting of {s['name']}",
)

# # comment in the line below to add the fit of each peak as its own gaussian line
fig_fit = plotly_plot(x=s['channel'], fig=fig_fit, fit_params=s['fit_params'], title=f"Gaussian fitting of {s['name']}")

fig_fit.show()

print("Check if the fit of the specified peaks are good in the plot here")

In [ ]:
# calculating the dispersion and offset with the two first peaks in s['peaks_channel'] and s['peaks_keV']
# then making the calibrated x axis, called s['kev_calibrated']

# it is important that the two first peaks in s['peaks_channel'] are corresponding to the two first peaks in s['peaks_keV']

# # comment in and change this if necessary
# s['peaks_keV'] = [1.098, 9.2517]

if s["peaks_keV"] is None:
    print(
        "ERROR: no peaks specified, please specify the peaks in the spectrum. No calibration was done."
    )
    print("Comment out the line above to set s['peaks_keV'] =[peak1, peak2] manually.")
    print(s["peaks_keV"])
else:
    calib = calibrate_channel_width_two_peaks(s["peaks_channel"], s["peaks_keV"])
    s["dispersion"] = calib[0]
    s["offset"] = calib[1]
    s["kev_calibrated"] = (s["channel"] - s["offset"]) * s["dispersion"]

In [ ]:
#  TODO: fjerne example_data
#  TODO: fjerne ubrukte datafiler
#  TODO: Bruke XRF_unknown_2nd.mca, ikke den andre

# printing a table with the amplitude, center position and energy width of the fitted peaks



print("Fitted peaks in keV:")
print(
    "Peak name | Peak position (mu) [keV] | Peak amplitude (a) | FWHM (std*2*sqrt(2*ln(2))) [keV] | Area under peak"
)
for i in range(len(s["fit_params"]) // 3):
    try:
        peak_name = s["peaks_names"][i]
    except (
        IndexError,
        TypeError,
    ):  # if s['peaks_names'] is None or not as long as the number of peaks
        peak_name = f"peak {i}"
    mu = channel_to_keV(s, value=s['fit_params'][i*3+1])
    amp = s['fit_params']
    fwhm = channel_to_keV(s, value=s['fit_params'][i * 3 + 2] * 2 * (np.log(2) * 2)**0.5, use_offset=False)
    area = area_under_peak(s['fit_params'][i*3+1], s['fit_params'][2 + 3 * i], s['fit_params'][3 * i])
    print(f"{peak_name:<9} | {mu:<24.4f} | {amp[i*3]:<18.4f} | {fwhm:<32.4f} | {area:.6f}")

print(f"\nFWHM defined as in: https://en.wikipedia.org/wiki/Full_width_at_half_maximum")

In [ ]:
# plotting the calibrated data

fig_calib = plotly_plot(
    x=s["kev_calibrated"],
    y_named=[s["intensity"], "intensity"],
    vlines=s["peaks_keV"],
    vlines_name=s["peaks_names"],
    title=f"Calibrated {s['name']}",
    xaxis_title="energy [keV], calibrated",
)

fig_calib.show()


# eventually saving the figure as eigther a interactive html file:
# open the html file in the browser, or click the button "Trust HTML" to view it in the notebook
fig_calib.write_html(f"plots/{s['filepath'].split('/')[-1].split('.')[0]}_calibrated.html")


# if you want to save the figure as a png file, click the "download plot as png" button in the plotly figure (camera icon)

# Saving the spectrum-dictionary to a json file

In [ ]:
# using the helper file saving_json.py
# Save/read the file to the folder Lab3_data_calibrated, adding the suffix _calibrated to the filename.
# NB! this will overwrite any existing file with the same name in the folder Lab3_data_calibrated

# save s
save_spectrum_to_json(s)

# read a saved spectrum
# s = read_saved_spectrum_from_json("Lab3_data_calibrated/[FILENAME].json")

# Unknown spectrum

## Using dispersion and offset from a calibrated spectrum to calibrate a unknown sample

now we use the fucntion init_unknown_spectrum_with_known(...)

In [ ]:
# We copy dispersion, offset, start_str, stop_str, line_endings, evt delimiter from s-dictionary to the new dictionary
# and set the new name and filepath

# s_un is the unknown spectrum


# # this is for the SEM_Cu file
s_un_SEM = init_unknown_spectrum_with_known(
    known_spectrum=s,
    name="SEM: Unknown",
    filepath="Lab3_data/SEM_unknown.msa",
)
s_un = s_un_SEM

# # The TEM_unknown er .msa, så vi må fjerne delimiter og endre stopstr og lineending to match .msa
# # delimiter må være False
# s_un_TEM = init_unknown_spectrum_with_known(
#     known_spectrum=s_TEM,
#     filepath="Lab3_data/TEM_unknown.msa",
#     name="TEM unknown",
#     delimiter=False,
#     line_endings=", \n",
#     start_str="#SPECTRUM    : Spectral Data Starts Here",
#     stop_str="#ENDOFDATA   : End Of Data and File",
# )
# s_un = s_un_TEM


# # # this is for XRF files
# s_un_XRF = init_unknown_spectrum_with_known(
#     known_spectrum=s_XRF,
#     name="XRF: Unknown",
#     filepath="Lab3_data/XRF_unknown.mca",
# )
# s_un = s_un_XRF


In [ ]:
# Now we plot the new and unknown spectrum

fig_unknown_plot = plotly_plot(
    x=s_un["kev_calibrated"],
    y_named=[s_un["intensity"], "TEM data"],
    vlines=s_un["peaks_keV"],
    vlines_name=s_un["peaks_names"],
    title=f"Calibrated {s_un['name']}",
    xaxis_title="keV calibrated",
)


fig_unknown_plot

## NB! In the code below you must change s['peaks_channel']

### Run the cell below to plot the intensity on channels, and then note down the peaks

In [ ]:
# We could also do fitting on the new plot by copying the steps from earlier

#
# # NB! This is not generally adapted! You must change the peaks_channel to match the peaks in the unknown spectrum
#

# peaks_keV is calculated from the fitting of the peaks_channel
# we want to fit on the channels
fig_unknown_plot_channels = plotly_plot(
    x=s_un["channel"],
    y_named=[s_un["intensity"], "x calibrated with Ga_Ka and Ga_La"],
    title=f"Plotted on channels: Calibrated {s_un['name']}",
)
fig_unknown_plot_channels.show()



#
# # # THIS VARIABLE peaks_channels ARE THE ONLY CODE YOU NEED TO CHANGE TO FIT THE PEAKS IN THE UNKNOWN SPECTRUM
#

# noted down the peaks in channel numbers
s_un["peaks_channel"] = [ 24, 31, 65, 170, 191, 202, 215, 472, 535] # this is for SEM_unknown
# s_un["peaks_channel"] = [380, 940] # this is for TEM_unknown



# do the fitting as we did earlier
fit_vals = fit_n_peaks_to_gaussian(
    x=s_un["channel"],
    y=s_un["intensity"],
    guessed_peaks=s_un["peaks_channel"],
)
s_un["fit_params"] = fit_vals[0]
s_un["fit_cov"] = fit_vals[1]
s_un["peaks_channel"] = s_un["fit_params"][1::3]
s_un["intensity_fit"] = n_gaussians(s_un["channel"], *fit_vals[0])

# print(f'Fitted peaks at: {s_un["peaks_channel"]}')

# now we scale down the fittet peaks with the dispersion and the offset
s_un["peaks_keV_fitted"] = (
    s_un["peaks_channel"] - s_un["offset"]
) * s_un["dispersion"]

# print(f'Fitted peaks in keV: {s_un["peaks_keV_fitted"]}')

#
# # NB! Because of the background, there will be some error on the fitting for some of the peaks!
#


# plotting the new spectrum with its fit and its lines in keV
fig_unknown_plot_fit = plotly_plot(
    x=s_un["kev_calibrated"],
    y=s_un["intensity"],
    y_fit=s_un["intensity_fit"],
    vlines=s_un["peaks_keV_fitted"],
    title=f"Calibrated {s_un['name']} on keV with fitted lines",
    xaxis_title="Calibrated keV",
)

# we can also plot each gaussian peak individually, but the plot looks a bit messy. Comment in to see it
# making the fit_params matrix in keV
# [amp1, mu1, sigma1, amp2, mu2, sigma2, ...]
# amp is amp
# (mu is mu - offset ) * dispersion
# sigma is sigma * dispersion
s_un["fit_params_keV"] = (s_un["fit_params"]).copy()
s_un["fit_params_keV"][1::3] = (
    s_un["fit_params_keV"][1::3] - s_un["offset"]
) * s_un[
    "dispersion"
]  # mu
s_un["fit_params_keV"][2::3] = (
    s_un["fit_params_keV"][2::3] * s_un["dispersion"]
)  # sigma

"""
# adding the single gaussian peaks to the plot
fig_unknown_plot_fit = plotly_plot(
    fig = fig_unknown_plot_fit,
    x=s_un["kev_calibrated"],
    fit_params=s_un["fit_params_keV"],
    title=f"Calibrated {s_un['name']} on keV with fitted lines",
    xaxis_title="Calibrated keV",
)
"""

fig_unknown_plot_fit.show()

## Identify the theoretical value of peaks

You can use Table 1-2 in the "X-ray data booklet" to find tabulated energies. Found here:

[http://web.mit.edu/8.13/www/JLExperiments/31/XBLx-raydatabook.pdf](http://web.mit.edu/8.13/www/JLExperiments/31/XBLx-raydatabook.pdf)

## Saving the unknown spectrum to a json file

In [ ]:
# again using the helper file saving_json.py
# NB! this will overwrite any existing file with the same name in the folder Lab3_data_calibrated

# eventually change the name of s_un to something more descriptive
print(f"Name of s_un now is: {s_un['name']}")

# # rename the spectrum
# s_un["name"] = "NEW NAME"


# save s_un
save_spectrum_to_json(s_un)



In [ ]:
# printing a table with the amplitude, center position and energy width of the fitted peaks



print("Fitted peaks in keV:")
print(
    "Peak name | Peak position (mu) [keV] | Peak amplitude (a) | FWHM (std*2*sqrt(2*ln(2))) [keV] | Area under peak"
)
for i in range(len(s_un["fit_params"]) // 3):
    try:
        peak_name = s_un["peaks_names"][i]
    except (
        IndexError,
        TypeError,
    ):  # if s_un['peaks_names'] is None or not as long as the number of peaks
        peak_name = f"peak {i}"
    mu = channel_to_keV(s, value=s_un['fit_params'][i*3+1])
    amp = s_un['fit_params']
    fwhm = channel_to_keV(s, value=s_un['fit_params'][i * 3 + 2] * 2 * (np.log(2) * 2)**0.5, use_offset=False)
    area = area_under_peak(s_un['fit_params'][i*3+1], s_un['fit_params'][2 + 3 * i], s_un['fit_params'][3 * i])
    print(f"{peak_name:<9} | {mu:<24.4f} | {amp[i*3]:<18.4f} | {fwhm:<32.4f} | {area:.6f}")

print(f"\nFWHM defined as in: https://en.wikipedia.org/wiki/Full_width_at_half_maximum")

In [ ]:
# eventually saving the figure as eigther a interactive html file:
# open the html file in the browser, or click the button "Trust HTML" to view it in the notebook
fig_calib.write_html(f"plots/{s_un['filepath'].split('/')[-1].split('.')[0]}_calibrated.html")


# if you want to save the figure as a png file, click the "download plot as png" button in the plotly figure (camera icon)

# New spectrum

## Save what you need from this calibration

### Eg save both or just one of the spectums to .json

### Eg save the plots as inteactive html (which you can get png from later)


# Now go up to loading of the files, and run the notebook again with either TEM or XRF or something else as s

**Remember to also change which file is read as the s_un**

In [ ]:
# PLotting two sprectra on top of each other, eg SEM and TEM

## this requires that the spectra have been calibrated, 
# ie do this after running through both eg SEM and TEM

try:
    fig_multiple_sources = plotly_plot(
        x=s_un_SEM["kev_calibrated"],
        y_named=[s_un_SEM["intensity"], "SEM data"],
        vlines=s_un_SEM["peaks_keV"],
        vlines_name=s_un_SEM["peaks_names"],
        xaxis_title="keV calibrated",
    )

    fig_multiple_sources = plotly_plot(
        x=s_un_TEM["kev_calibrated"],
        y_named=[s_un_TEM["intensity"], "TEM data"],
        title=f"Calibrated SEM and TEM",
        xaxis_title="keV calibrated",
        fig = fig_multiple_sources
    )


    fig_multiple_sources.show()
except NameError:
    print("SEM and TEM spectra not found, skipping plotting of multiple sources")
    print("You need to run the calibration for both SEM and TEM spectra to plot them together")

fig_multiple_sources.write_html(f"plots/SEM_and_TEM_unknown_together_calibrated.html")

In [ ]:
## can you identify the difference between the two spectra?
### Hint: overvoltage, strays, etc

In [1]:
# Run this cell if you want to save all the files in a zip file.
# You can then download the zip file from the file browser on the left.

import zipfile, os

zip_filename = "download.zip"

# if there is a zip file with the same name, delete it
if os.path.exists(zip_filename):
    os.remove(zip_filename)

# create a zip file
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zip:
    for root, dirs, files in os.walk("."):
        for file in files:
            if os.path.normpath(os.path.join(root,file)) == os.path.normpath(zip_filename):
                continue # Skip zip_filename
            zip.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), "."))

# download the zip file to your computer by right clicking on the file in the file browser on the left